## Transforming Climata and Housing Data

In [3]:
import pandas as pd

In [30]:
df_weather = pd.read_csv('../webscrapers/weather_stations.csv', parse_dates=['yearmo'])
df_weather.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_20120\2806954434.py:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv('../webscrapers/weather_stations.csv', parse_dates=['yearmo'])


,Long,Lat,Stn_Name,Clim_ID,Prov_or_Ter,Tm,DwTm,D,Tx,DwTx,...,DwP,P%N,S_G,Pd,BS,DwBS,BS%,HDD,CDD,yearmo
0,-123.742,48.935,CHEMAINUS,1011500,BC,3.8,0.0,NaN,10.5,0.0,...,0.0,NaN,0.0,21.0,NaN,NaN,NaN,439.8,0.0,201201
1,-124.133,48.824,COWICHAN LAKE FORESTRY,1012040,BC,2.2,0.0,-1.2,12.5,0.0,...,0.0,114.0,0.0,22.0,NaN,NaN,NaN,489.7,0.0,201201
2,-124.052,48.829,LAKE COWICHAN,1012055,BC,2.1,14.0,-1.4,11.5,9.0,...,9.0,NaN,0.0,15.0,NaN,NaN,NaN,269.5,0.0,201201
3,-123.225,48.424,DISCOVERY ISLAND,1012475,BC,5.2,0.0,NaN,11.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,397.1,0.0,201201
4,-123.728,48.735,DUNCAN KELVIN CREEK,1012573,BC,2.7,0.0,-0.9,11.5,0.0,...,0.0,105.0,0.0,19.0,NaN,NaN,NaN,475.2,0.0,201201


In [31]:
#fixing date pattern
df_weather['date']=df_weather['yearmo'].str[:4]+'-'+df_weather['yearmo'].str[4:]
df_weather['date'] = pd.to_datetime(df_weather.date)
df_weather.head()

,Long,Lat,Stn_Name,Clim_ID,Prov_or_Ter,Tm,DwTm,D,Tx,DwTx,...,P%N,S_G,Pd,BS,DwBS,BS%,HDD,CDD,yearmo,date
0,-123.742,48.935,CHEMAINUS,1011500,BC,3.8,0.0,NaN,10.5,0.0,...,NaN,0.0,21.0,NaN,NaN,NaN,439.8,0.0,201201,2012-01-01
1,-124.133,48.824,COWICHAN LAKE FORESTRY,1012040,BC,2.2,0.0,-1.2,12.5,0.0,...,114.0,0.0,22.0,NaN,NaN,NaN,489.7,0.0,201201,2012-01-01
2,-124.052,48.829,LAKE COWICHAN,1012055,BC,2.1,14.0,-1.4,11.5,9.0,...,NaN,0.0,15.0,NaN,NaN,NaN,269.5,0.0,201201,2012-01-01
3,-123.225,48.424,DISCOVERY ISLAND,1012475,BC,5.2,0.0,NaN,11.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,397.1,0.0,201201,2012-01-01
4,-123.728,48.735,DUNCAN KELVIN CREEK,1012573,BC,2.7,0.0,-0.9,11.5,0.0,...,105.0,0.0,19.0,NaN,NaN,NaN,475.2,0.0,201201,2012-01-01


In [32]:
#Selecting important features in data
df_weather = df_weather[['Stn_Name','Clim_ID','Prov_or_Ter','Tm','Tx','Tn','P','S_G','date']]
df_weather.head()

,Stn_Name,Clim_ID,Prov_or_Ter,Tm,Tx,Tn,P,S_G,date
0,CHEMAINUS,1011500,BC,3.8,10.5,-5.0,240.6,0.0,2012-01-01
1,COWICHAN LAKE FORESTRY,1012040,BC,2.2,12.5,-7.5,408.1,0.0,2012-01-01
2,LAKE COWICHAN,1012055,BC,2.1,11.5,-8.5,402.2,0.0,2012-01-01
3,DISCOVERY ISLAND,1012475,BC,5.2,11.6,-5.5,NaN,NaN,2012-01-01
4,DUNCAN KELVIN CREEK,1012573,BC,2.7,11.5,-7.5,260.3,0.0,2012-01-01


In [33]:
df_weather.isnull().sum()

Stn_Name            0
Clim_ID             0
Prov_or_Ter         0
Tm              13149
Tx               7896
Tn               8032
P               20378
S_G            104508
date                0
dtype: int64

In [34]:
#dropping rows with ####### useless values
df_weather = df_weather[df_weather.P != '######'].copy()
df_weather['P'] = df_weather['P'].astype(float)

In [35]:
#filing nan values with column averages
df_weather['S_G'].fillna((df_weather['S_G'].mean()), inplace=True)
df_weather['Tm'].fillna((df_weather['Tm'].mean()), inplace=True)
df_weather['Tx'].fillna((df_weather['Tx'].mean()), inplace=True)
df_weather['Tn'].fillna((df_weather['Tn'].mean()), inplace=True)
df_weather['P'].fillna((df_weather['P'].astype(float).mean()), inplace=True)
df_weather

,Stn_Name,Clim_ID,Prov_or_Ter,Tm,Tx,Tn,P,S_G,date
0,CHEMAINUS,1011500,BC,3.8,10.5,-5.0,240.600000,0.000000,2012-01-01
1,COWICHAN LAKE FORESTRY,1012040,BC,2.2,12.5,-7.5,408.100000,0.000000,2012-01-01
2,LAKE COWICHAN,1012055,BC,2.1,11.5,-8.5,402.200000,0.000000,2012-01-01
3,DISCOVERY ISLAND,1012475,BC,5.2,11.6,-5.5,60.910173,12.917683,2012-01-01
4,DUNCAN KELVIN CREEK,1012573,BC,2.7,11.5,-7.5,260.300000,0.000000,2012-01-01
...,...,...,...,...,...,...,...,...,...
178651,MAKKOVIK A,8502580,NL,-20.0,0.0,-32.7,60.910173,12.917683,2023-02-01
178652,MARY'S HARBOUR A,8502592,NL,-16.4,1.8,-31.7,3.100000,12.917683,2023-02-01
178653,NAIN,8502799,NL,-21.6,-0.4,-33.4,60.910173,12.917683,2023-02-01
178654,NAIN A,8502810,NL,-21.8,-1.3,-33.7,60.910173,12.917683,2023-02-01


In [36]:
#Grouping by location and date to create regional averages
regional_mean=df_weather.groupby(['Prov_or_Ter','date'])[['Tm','Tx','Tn','P','S_G']].mean()
regional_mean=regional_mean.reset_index()
regional_mean.rename(columns={'Tm': 'Tm Average','Prov_or_Ter':'location', 'Tx': 'Tx Average', 'Tn': 'Tn Average',  'P': 'P Average',  'S_G': 'S_G Average'}, inplace=True)
regional_mean

,location,date,Tm Average,Tx Average,Tn Average,P Average,S_G Average
0,AB,2012-01-01,-6.923002,11.398783,-33.632209,12.465167,11.258989
1,AB,2012-02-01,-6.292396,8.171374,-24.513714,13.758041,12.343840
2,AB,2012-03-01,-0.904317,14.305485,-18.870144,21.922648,10.703824
3,AB,2012-04-01,4.395967,22.516142,-9.429845,38.454894,10.295377
4,AB,2012-05-01,9.505343,26.094101,-3.233266,49.041344,10.572356
...,...,...,...,...,...,...,...
1737,YT,2022-10-01,-1.275000,15.710714,-18.221429,57.730450,11.235736
1738,YT,2022-11-01,-13.167857,2.596429,-35.492857,40.234385,15.813045
1739,YT,2022-12-01,-22.457143,-0.689286,-44.425000,43.277242,21.384473
1740,YT,2023-01-01,-14.571429,-0.164286,-28.867857,42.570099,26.208842


In [38]:
regional_mean.to_csv("regional_weather_summaries.csv", index=False)

In [40]:
#Importing second dataset
housing = pd.read_csv('../webscrapers/structure_housing_data_22-Mar-23_08-30PM.csv')
housing.head()

,location,date,total,house,land
0,Canada,January 2012,90.8,89.4,93.8
1,Canada,February 2012,91.0,89.6,93.9
2,Canada,March 2012,91.3,89.9,94.1
3,Canada,April 2012,91.5,90.2,94.2
4,Canada,May 2012,91.7,90.4,94.4


In [41]:
regional_mean.location.unique()

array(['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC',
       'SK', 'YT'], dtype=object)

In [42]:
#changing locations to acronyms to match other dataset
location_mapping = {
    'Canada':"CA",
    'Newfoundland and Labrador':'NL', 
    'Prince Edward Island':"PE",
    'Nova Scotia':'NL',
    'New Brunswick':'NB', 
    'Quebec':'QC', 
    'Ontario':'ON', 
    'Manitoba':'MB',
    'Saskatchewan':'SK',
    'Alberta':'AB',
    'British Columbia':'BC'
}

housing["location"] = housing.location.apply(lambda x: location_mapping[x])

In [43]:
#standardizing date format
housing['date'] = pd.to_datetime(housing.date)
housing

,location,date,total,house,land
0,CA,2012-01-01,90.8,89.4,93.8
1,CA,2012-02-01,91.0,89.6,93.9
2,CA,2012-03-01,91.3,89.9,94.1
3,CA,2012-04-01,91.5,90.2,94.2
4,CA,2012-05-01,91.7,90.4,94.4
...,...,...,...,...,...
1469,BC,2022-10-01,127.8,127.6,120.4
1470,BC,2022-11-01,127.7,127.5,120.3
1471,BC,2022-12-01,127.6,127.4,120.2
1472,BC,2023-01-01,127.1,126.8,119.9


In [44]:
#Saving to CSV
housing.to_csv("real_estate.csv", index=False)

In [49]:
#merging 
new_result = pd.merge(regional_mean, housing, how="left", on=["location","date"])
new_result.isnull().sum()
new_result

,location,date,Tm Average,Tx Average,Tn Average,P Average,S_G Average,total,house,land
0,AB,2012-01-01,-6.923002,11.398783,-33.632209,12.465167,11.258989,93.2,93.2,93.2
1,AB,2012-02-01,-6.292396,8.171374,-24.513714,13.758041,12.343840,93.2,93.1,93.4
2,AB,2012-03-01,-0.904317,14.305485,-18.870144,21.922648,10.703824,93.6,93.6,93.5
3,AB,2012-04-01,4.395967,22.516142,-9.429845,38.454894,10.295377,93.7,93.8,93.8
4,AB,2012-05-01,9.505343,26.094101,-3.233266,49.041344,10.572356,93.9,93.9,93.9
...,...,...,...,...,...,...,...,...,...,...
1871,YT,2022-10-01,-1.275000,15.710714,-18.221429,57.730450,11.235736,NaN,NaN,NaN
1872,YT,2022-11-01,-13.167857,2.596429,-35.492857,40.234385,15.813045,NaN,NaN,NaN
1873,YT,2022-12-01,-22.457143,-0.689286,-44.425000,43.277242,21.384473,NaN,NaN,NaN
1874,YT,2023-01-01,-14.571429,-0.164286,-28.867857,42.570099,26.208842,NaN,NaN,NaN


In [50]:
#creating surrogate keys
new_result['Surrogate Keys'] = range(1,len(new_result)+1)
new_result = new_result.reindex(columns=['Surrogate Keys'] + list([c for c in new_result.columns if c!= 'Surrogate Keys']))

In [ ]:
fact_table = new_result[new_result['total'].notna()]

fact_table.to_csv("fact_table.csv", index=False)
fact_table.head()

In [54]:
location_dimension = fact_table[['location']].copy()
location_dimension = location_dimension.drop_duplicates().reset_index(drop=True)
location_dimension.to_csv("location_dimension.csv", index=False)
location_dimension

,location
0,AB
1,BC
2,MB
3,NB
4,NL
5,ON
6,PE
7,QC
8,SK


In [58]:
date_dimension = fact_table[['date']].copy()
date_dimension['Month'] = date_dimension['date'].dt.month 
date_dimension['Year'] = date_dimension['date'].dt.year

# date_dimension = date_dimension.drop(['date'], axis=1)

date_dimension.to_csv("date_dimension.csv")
#date dimension

In [59]:
#weather station dimension
df_weather.to_csv("weather_station_summary.csv")
df_weather

,Stn_Name,Clim_ID,Prov_or_Ter,Tm,Tx,Tn,P,S_G,date
0,CHEMAINUS,1011500,BC,3.8,10.5,-5.0,240.600000,0.000000,2012-01-01
1,COWICHAN LAKE FORESTRY,1012040,BC,2.2,12.5,-7.5,408.100000,0.000000,2012-01-01
2,LAKE COWICHAN,1012055,BC,2.1,11.5,-8.5,402.200000,0.000000,2012-01-01
3,DISCOVERY ISLAND,1012475,BC,5.2,11.6,-5.5,60.910173,12.917683,2012-01-01
4,DUNCAN KELVIN CREEK,1012573,BC,2.7,11.5,-7.5,260.300000,0.000000,2012-01-01
...,...,...,...,...,...,...,...,...,...
178651,MAKKOVIK A,8502580,NL,-20.0,0.0,-32.7,60.910173,12.917683,2023-02-01
178652,MARY'S HARBOUR A,8502592,NL,-16.4,1.8,-31.7,3.100000,12.917683,2023-02-01
178653,NAIN,8502799,NL,-21.6,-0.4,-33.4,60.910173,12.917683,2023-02-01
178654,NAIN A,8502810,NL,-21.8,-1.3,-33.7,60.910173,12.917683,2023-02-01
